In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import CategoricalCrossentropy , mae,mse
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import cv2
import os
import pandas
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import pandas as pd

In [2]:
df=pandas.read_csv("train.csv")
df

,matchId,playerId,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second,outcome
0,m_91,p_103,جریان بازی,پای راست,13.47,-11.22,1,0,متوسط,70,9,گُل
1,m_17,p_16,جریان بازی,پای چپ,9.48,14.22,3,0,متوسط,55,4,مهار توسط دروازه بان
2,m_111,p_88,ضربه آزاد مستقیم,پای چپ,29.43,-1.25,6,2,کم,86,31,مهار توسط دروازه بان
3,m_142,p_87,جریان بازی,پای راست,26.93,1.00,4,1,متوسط,77,2,موقعیت از دست رفته
4,m_117,p_9,جریان بازی,پای راست,10.72,5.24,2,0,متوسط,76,46,گُل
...,...,...,...,...,...,...,...,...,...,...,...,...
8920,m_57,p_115,جریان بازی,سر,6.48,3.99,3,0,زیاد,69,50,موقعیت از دست رفته
8921,m_59,p_76,جریان بازی,پای راست,21.45,-8.73,4,1,متوسط,15,53,برخورد به دفاع
8922,m_55,p_150,جریان بازی,پای چپ,11.97,3.24,3,0,متوسط,84,34,موقعیت از دست رفته
8923,m_33,p_130,جریان بازی,پای راست,6.48,-6.98,1,0,زیاد,4,39,موقعیت از دست رفته


In [3]:
df.columns

Index(['matchId', 'playerId', 'playType', 'bodyPart', 'x', 'y',
       'interveningOpponents', 'interveningTeammates', 'interferenceOnShooter',
       'minute', 'second', 'outcome'],
      dtype='object')

In [4]:
# df[df.isna().any(axis=1)]

In [5]:
df_weighted=pandas.concat([df,df[df.outcome=="گُل"] ],ignore_index=True) 
df_weighted.outcome=df_weighted.outcome.map(lambda result: "گل نه" if result!="گُل" else "گل")
df_weighted

,matchId,playerId,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second,outcome
0,m_91,p_103,جریان بازی,پای راست,13.47,-11.22,1,0,متوسط,70,9,گل
1,m_17,p_16,جریان بازی,پای چپ,9.48,14.22,3,0,متوسط,55,4,گل نه
2,m_111,p_88,ضربه آزاد مستقیم,پای چپ,29.43,-1.25,6,2,کم,86,31,گل نه
3,m_142,p_87,جریان بازی,پای راست,26.93,1.00,4,1,متوسط,77,2,گل نه
4,m_117,p_9,جریان بازی,پای راست,10.72,5.24,2,0,متوسط,76,46,گل
...,...,...,...,...,...,...,...,...,...,...,...,...
10001,m_119,p_77,جریان بازی,پای راست,8.23,2.74,2,0,متوسط,80,19,گل
10002,m_221,p_155,جریان بازی,پای راست,2.49,-1.00,1,0,کم,30,6,گل
10003,m_132,p_18,جریان بازی,پای چپ,16.71,10.23,3,0,کم,37,11,گل
10004,m_146,p_6,جریان بازی,پای چپ,7.98,7.98,3,1,متوسط,18,47,گل


In [7]:
df.dtypes

matchId                   object
playerId                  object
playType                  object
bodyPart                  object
x                        float64
y                        float64
interveningOpponents       int64
interveningTeammates       int64
interferenceOnShooter     object
minute                     int64
second                     int64
outcome                   object
dtype: object

In [8]:
string_columns=df_weighted.select_dtypes(include=['object']).columns.tolist()
string_columns.remove("matchId")
string_columns.remove("playerId")
string_columns, len(string_columns)

(['playType', 'bodyPart', 'interferenceOnShooter', 'outcome'], 4)

In [9]:
# label_encoders=[]
# for str_col in string_columns:
#     label_encoder = LabelEncoder()
#     df_weighted[str_col] = label_encoder.fit_transform(df_weighted[str_col])
#     label_encoders.append(label_encoder)

In [10]:
df_one_hot=df_weighted.copy()
if "outcome" in string_columns:
    string_columns.remove("outcome")
for str_col in string_columns:
    print(str_col)
    one_hot_encoded = pd.get_dummies(df_weighted[str_col], prefix=str_col)
    df_one_hot=pd.concat([df_one_hot, one_hot_encoded], axis=1)
    df_one_hot.drop(str_col,axis=1,inplace=True)

playType
bodyPart
interferenceOnShooter


In [11]:
df_weighted[df_weighted.matchId=="m_153" ][df_weighted.playerId=="p_22"]

C:\Users\Parsa\AppData\Local\Temp\ipykernel_2088\3564173349.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_weighted[df_weighted.matchId=="m_153" ][df_weighted.playerId=="p_22"]


,matchId,playerId,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second,outcome
434,m_153,p_22,جریان بازی,پای راست,9.23,-1.25,1,0,NaN,33,48,گل نه


In [12]:
df_weighted=df_one_hot

In [13]:
df_weighted.columns

Index(['matchId', 'playerId', 'x', 'y', 'interveningOpponents',
       'interveningTeammates', 'minute', 'second', 'outcome',
       'playType_جریان بازی', 'playType_ضربه آزاد مستقیم',
       'playType_مستقیم از کرنر', 'playType_پنالتی', 'bodyPart_سایر',
       'bodyPart_سر', 'bodyPart_پای راست', 'bodyPart_پای چپ',
       'interferenceOnShooter_زیاد', 'interferenceOnShooter_متوسط',
       'interferenceOnShooter_کم'],
      dtype='object')

In [14]:
#remember to remove minute and second if necessary
X=df_weighted.drop(['matchId',"playerId","outcome"], axis=1).to_numpy()
X.shape

(10006, 17)

In [22]:
df.isna().sum().sum()

34

In [27]:
import numpy as np
from sklearn.svm import OneClassSVM

# Create a sample dataset
data = np.random.randn(100, 2)  # 100 samples, 2 features

# Create a OneClassSVM instance
outlier_detector = OneClassSVM(nu=0.05)  # Nu is an upper bound on the proportion of outliers

# Fit the model and predict outliers
outliers = outlier_detector.fit_predict(X_normalized)

print("Predicted outliers:",X_normalized[outliers==-1].shape)

Predicted outliers: (496, 17)


In [31]:
X_normalized[outliers==1].shape

(9510, 17)

In [28]:
X_normalized.shape

(10006, 17)

In [15]:
X_normalized = np.empty_like(X, dtype=float)
scalers = []
for feature_index in range(X.shape[1]):
    scaler = MinMaxScaler()
    feature = X[:, feature_index].reshape(-1, 1)
    scaler.fit(feature)
    X_normalized[:, feature_index] = scaler.transform(feature).flatten()
    scalers.append(scaler)

print("Original Matrix X:")
print(X)
print("\nNormalized Matrix X:")
print(X_normalized)

Original Matrix X:
[[13.47 -11.22 1 ... False True False]
 [9.48 14.22 3 ... False True False]
 [29.43 -1.25 6 ... False False True]
 ...
 [16.71 10.23 3 ... False False True]
 [7.98 7.98 3 ... False True False]
 [9.98 6.48 2 ... False True False]]

Normalized Matrix X:
[[0.19020051 0.33461085 0.09090909 ... 0.         1.         0.        ]
 [0.13386049 0.70961085 0.27272727 ... 0.         1.         0.        ]
 [0.41556058 0.48157429 0.54545455 ... 0.         0.         1.        ]
 ...
 [0.2359503  0.65079599 0.27272727 ... 0.         0.         1.        ]
 [0.11268003 0.61762972 0.27272727 ... 0.         1.         0.        ]
 [0.14092064 0.59551887 0.18181818 ... 0.         1.         0.        ]]


In [16]:
y=df_weighted.outcome.to_numpy()
y.shape
y=np.vectorize(lambda result: 0 if result!="گل" else 1) (y)

In [17]:
trainX, crossX, trainY, crossY = train_test_split(X_normalized,y, test_size=0.10,random_state=42)

In [123]:
model=Sequential([
    Input( shape=(trainX.shape[1],) ),
    Dense(256,activation="relu"),
     Dense(128,activation="relu"),
    BatchNormalization(),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss="binary_crossentropy",metrics=["accuracy"])

In [124]:
save_path="./result.hdf5"
checkpoint = ModelCheckpoint(save_path, monitor="val_accuracy",
	save_best_only=True, verbose=1)
callbacks = [checkpoint]
model.fit(trainX,trainY,epochs=500
          ,validation_data=(crossX, crossY)
          ,callbacks=callbacks,batch_size=32
         )

Epoch 1/500
282/282 [==============================] - ETA: 0s - loss: 0.4659 - accuracy: 0.8000
Epoch 1: val_accuracy improved from -inf to 0.81419, saving model to .\result.hdf5
282/282 [==============================] - 6s 11ms/step - loss: 0.4659 - accuracy: 0.8000 - val_loss: 0.4810 - val_accuracy: 0.8142
Epoch 2/500
280/282 [============================>.] - ETA: 0s - loss: 0.4406 - accuracy: 0.8058
Epoch 2: val_accuracy improved from 0.81419 to 0.81518, saving model to .\result.hdf5
282/282 [==============================] - 3s 10ms/step - loss: 0.4407 - accuracy: 0.8059 - val_loss: 0.4309 - val_accuracy: 0.8152
Epoch 3/500
281/282 [============================>.] - ETA: 0s - loss: 0.4369 - accuracy: 0.8057
Epoch 3: val_accuracy improved from 0.81518 to 0.81618, saving model to .\result.hdf5
282/282 [==============================] - 3s 10ms/step - loss: 0.4368 - accuracy: 0.8059 - val_loss: 0.4250 - val_accuracy: 0.8162
Epoch 4/500
278/282 [============================>.] - ETA

282/282 [==============================] - 2s 9ms/step - loss: 0.3723 - accuracy: 0.8298 - val_loss: 0.3996 - val_accuracy: 0.8202
Epoch 30/500
279/282 [============================>.] - ETA: 0s - loss: 0.3687 - accuracy: 0.8292
Epoch 30: val_accuracy did not improve from 0.83716
282/282 [==============================] - 2s 8ms/step - loss: 0.3699 - accuracy: 0.8285 - val_loss: 0.4045 - val_accuracy: 0.8062
Epoch 31/500
281/282 [============================>.] - ETA: 0s - loss: 0.3639 - accuracy: 0.8331
Epoch 31: val_accuracy did not improve from 0.83716
282/282 [==============================] - 2s 9ms/step - loss: 0.3640 - accuracy: 0.8331 - val_loss: 0.4103 - val_accuracy: 0.8332
Epoch 32/500
280/282 [============================>.] - ETA: 0s - loss: 0.3616 - accuracy: 0.8337
Epoch 32: val_accuracy did not improve from 0.83716
282/282 [==============================] - 2s 9ms/step - loss: 0.3614 - accuracy: 0.8338 - val_loss: 0.4005 - val_accuracy: 0.8242
Epoch 33/500
281/282 [====

Epoch 59/500
279/282 [============================>.] - ETA: 0s - loss: 0.3021 - accuracy: 0.8560
Epoch 59: val_accuracy did not improve from 0.83716
282/282 [==============================] - 2s 8ms/step - loss: 0.3023 - accuracy: 0.8560 - val_loss: 0.3743 - val_accuracy: 0.8252
Epoch 60/500
280/282 [============================>.] - ETA: 0s - loss: 0.3071 - accuracy: 0.8561
Epoch 60: val_accuracy did not improve from 0.83716
282/282 [==============================] - 2s 8ms/step - loss: 0.3070 - accuracy: 0.8562 - val_loss: 0.3843 - val_accuracy: 0.8152
Epoch 61/500
276/282 [============================>.] - ETA: 0s - loss: 0.3064 - accuracy: 0.8572
Epoch 61: val_accuracy did not improve from 0.83716
282/282 [==============================] - 2s 8ms/step - loss: 0.3054 - accuracy: 0.8577 - val_loss: 0.3874 - val_accuracy: 0.8172
Epoch 62/500
275/282 [============================>.] - ETA: 0s - loss: 0.3029 - accuracy: 0.8584
Epoch 62: val_accuracy did not improve from 0.83716
282/282

Epoch 117/500
279/282 [============================>.] - ETA: 0s - loss: 0.2132 - accuracy: 0.9017
Epoch 117: val_accuracy improved from 0.85015 to 0.85315, saving model to .\result.hdf5
282/282 [==============================] - 3s 9ms/step - loss: 0.2130 - accuracy: 0.9017 - val_loss: 0.3606 - val_accuracy: 0.8531
Epoch 118/500
278/282 [============================>.] - ETA: 0s - loss: 0.2182 - accuracy: 0.8996
Epoch 118: val_accuracy did not improve from 0.85315
282/282 [==============================] - 2s 8ms/step - loss: 0.2181 - accuracy: 0.8995 - val_loss: 0.3835 - val_accuracy: 0.8142
Epoch 119/500
277/282 [============================>.] - ETA: 0s - loss: 0.2170 - accuracy: 0.8980
Epoch 119: val_accuracy did not improve from 0.85315
282/282 [==============================] - 2s 8ms/step - loss: 0.2164 - accuracy: 0.8986 - val_loss: 0.3695 - val_accuracy: 0.8322
Epoch 120/500
278/282 [============================>.] - ETA: 0s - loss: 0.2163 - accuracy: 0.8994
Epoch 120: val_ac

Epoch 146/500
275/282 [============================>.] - ETA: 0s - loss: 0.1933 - accuracy: 0.9093
Epoch 146: val_accuracy did not improve from 0.85315
282/282 [==============================] - 2s 8ms/step - loss: 0.1957 - accuracy: 0.9092 - val_loss: 0.4126 - val_accuracy: 0.8272
Epoch 147/500
282/282 [==============================] - ETA: 0s - loss: 0.1975 - accuracy: 0.9117
Epoch 147: val_accuracy did not improve from 0.85315
282/282 [==============================] - 2s 8ms/step - loss: 0.1975 - accuracy: 0.9117 - val_loss: 0.3712 - val_accuracy: 0.8422
Epoch 148/500
281/282 [============================>.] - ETA: 0s - loss: 0.2011 - accuracy: 0.9086
Epoch 148: val_accuracy did not improve from 0.85315
282/282 [==============================] - 2s 8ms/step - loss: 0.2013 - accuracy: 0.9086 - val_loss: 0.3733 - val_accuracy: 0.8482
Epoch 149/500
279/282 [============================>.] - ETA: 0s - loss: 0.1926 - accuracy: 0.9143
Epoch 149: val_accuracy did not improve from 0.85315

...

Epoch 349/500
281/282 [============================>.] - ETA: 0s - loss: 0.1194 - accuracy: 0.9511
Epoch 349: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 8ms/step - loss: 0.1193 - accuracy: 0.9511 - val_loss: 0.4522 - val_accuracy: 0.8611
Epoch 350/500
276/282 [============================>.] - ETA: 0s - loss: 0.1164 - accuracy: 0.9528
Epoch 350: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 9ms/step - loss: 0.1166 - accuracy: 0.9528 - val_loss: 0.4196 - val_accuracy: 0.8791
Epoch 351/500
276/282 [============================>.] - ETA: 0s - loss: 0.1054 - accuracy: 0.9569
Epoch 351: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 8ms/step - loss: 0.1049 - accuracy: 0.9570 - val_loss: 0.4729 - val_accuracy: 0.8661
Epoch 352/500
277/282 [============================>.] - ETA: 0s - loss: 0.1063 - accuracy: 0.9556
Epoch 352: val_accuracy did not improve from 0.89011

Epoch 378/500
277/282 [============================>.] - ETA: 0s - loss: 0.1226 - accuracy: 0.9501
Epoch 378: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 8ms/step - loss: 0.1218 - accuracy: 0.9502 - val_loss: 0.4540 - val_accuracy: 0.8751
Epoch 379/500
276/282 [============================>.] - ETA: 0s - loss: 0.1067 - accuracy: 0.9565
Epoch 379: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 8ms/step - loss: 0.1062 - accuracy: 0.9569 - val_loss: 0.4410 - val_accuracy: 0.8891
Epoch 380/500
276/282 [============================>.] - ETA: 0s - loss: 0.0994 - accuracy: 0.9595
Epoch 380: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 8ms/step - loss: 0.0987 - accuracy: 0.9598 - val_loss: 0.4597 - val_accuracy: 0.8811
Epoch 381/500
279/282 [============================>.] - ETA: 0s - loss: 0.1033 - accuracy: 0.9572
Epoch 381: val_accuracy did not improve from 0.89011

Epoch 407/500
280/282 [============================>.] - ETA: 0s - loss: 0.0921 - accuracy: 0.9634
Epoch 407: val_accuracy did not improve from 0.89011
282/282 [==============================] - 3s 9ms/step - loss: 0.0923 - accuracy: 0.9634 - val_loss: 0.4954 - val_accuracy: 0.8681
Epoch 408/500
280/282 [============================>.] - ETA: 0s - loss: 0.0979 - accuracy: 0.9609
Epoch 408: val_accuracy did not improve from 0.89011
282/282 [==============================] - 3s 9ms/step - loss: 0.0982 - accuracy: 0.9608 - val_loss: 0.5152 - val_accuracy: 0.8731
Epoch 409/500
278/282 [============================>.] - ETA: 0s - loss: 0.1092 - accuracy: 0.9550
Epoch 409: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 9ms/step - loss: 0.1087 - accuracy: 0.9555 - val_loss: 0.4764 - val_accuracy: 0.8861
Epoch 410/500
276/282 [============================>.] - ETA: 0s - loss: 0.0935 - accuracy: 0.9608
Epoch 410: val_accuracy did not improve from 0.89011

Epoch 436/500
278/282 [============================>.] - ETA: 0s - loss: 0.0940 - accuracy: 0.9628
Epoch 436: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 9ms/step - loss: 0.0944 - accuracy: 0.9627 - val_loss: 0.4806 - val_accuracy: 0.8831
Epoch 437/500
277/282 [============================>.] - ETA: 0s - loss: 0.0918 - accuracy: 0.9634
Epoch 437: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 9ms/step - loss: 0.0915 - accuracy: 0.9636 - val_loss: 0.5038 - val_accuracy: 0.8811
Epoch 438/500
280/282 [============================>.] - ETA: 0s - loss: 0.0935 - accuracy: 0.9627
Epoch 438: val_accuracy did not improve from 0.89011
282/282 [==============================] - 2s 9ms/step - loss: 0.0932 - accuracy: 0.9628 - val_loss: 0.5260 - val_accuracy: 0.8791
Epoch 439/500
280/282 [============================>.] - ETA: 0s - loss: 0.0953 - accuracy: 0.9610
Epoch 439: val_accuracy did not improve from 0.89011

KeyboardInterrupt: 

In [128]:
loaded_model=tf.keras.models.load_model(save_path)

In [139]:
y_val_preds=loaded_model.predict(crossX)>=0.5

32/32 [==============================] - 0s 2ms/step


In [140]:
print(classification_report(crossY,y_val_preds))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93       794
           1       0.69      0.84      0.76       207

    accuracy                           0.89      1001
   macro avg       0.82      0.87      0.84      1001
weighted avg       0.90      0.89      0.89      1001



In [141]:
#remember to NORMALIZE DATA before prediction
df_test=pandas.read_csv("test.csv") #change this to test.csv
new_order=['playType', 'bodyPart', 'x', 'y',
        'interveningOpponents', 'interveningTeammates', 'interferenceOnShooter',
         'minute', 'second']
df_test=df_test[new_order]
df_test

,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second
0,جریان بازی,سر,6.98,-5.49,1,0,کم,76,7
1,جریان بازی,پای چپ,27.43,1.00,2,1,کم,81,18
2,جریان بازی,پای چپ,13.47,-1.00,4,0,متوسط,86,50
3,جریان بازی,پای راست,9.23,-9.23,1,0,زیاد,69,39
4,جریان بازی,پای راست,10.72,-8.48,2,1,کم,53,15
...,...,...,...,...,...,...,...,...,...
1995,جریان بازی,پای راست,7.23,12.47,1,0,کم,86,21
1996,جریان بازی,پای چپ,11.22,8.48,2,0,کم,11,19
1997,جریان بازی,پای راست,11.47,-1.25,2,0,متوسط,8,33
1998,جریان بازی,پای راست,18.45,-14.71,4,0,متوسط,53,42


In [189]:
df_test.isnull().sum()

playType                 0
bodyPart                 0
x                        0
y                        0
interveningOpponents     0
interveningTeammates     0
interferenceOnShooter    9
minute                   0
second                   0
dtype: int64

In [144]:
string_columns

['playType', 'bodyPart', 'interferenceOnShooter']

In [146]:
df_test_one_hot=df_test.copy()
if "outcome" in string_columns:
    string_columns.remove("outcome")

for str_col in string_columns:
    print(str_col)
    one_hot_encoded = pd.get_dummies(df_test_one_hot[str_col], prefix=str_col)
    df_test_one_hot=pd.concat([df_test_one_hot, one_hot_encoded], axis=1)
    df_test_one_hot.drop(str_col,axis=1,inplace=True)

playType
bodyPart
interferenceOnShooter


In [145]:
# for i,str_col in enumerate(string_columns):
#     label_encoder = label_encoders[i]
#     df_test[str_col] = label_encoder.transform(df_test[str_col])


IndexError: list index out of range

In [189]:
# X_test=df_test.to_numpy()
# X_test.shape

(2000, 9)

In [166]:
X_test=df_test_one_hot.to_numpy()
X_test.shape

(2000, 17)

In [170]:
X_test_normalized = np.empty_like(X_test, dtype=float)
for feature_index in range(X_test.shape[1]):
        scaler=scalers[feature_index]
        feature = X_test[:, feature_index].reshape(-1, 1)
        X_test_normalized[:, feature_index] = scaler.transform(feature).flatten()
X_test_normalized.shape

(2000, 17)

In [186]:
preds=loaded_model.predict(X_test_normalized)
print(preds)

63/63 [==============================] - 0s 2ms/step
[[5.0290156e-04]
 [3.3839112e-07]
 [6.5448850e-02]
 ...
 [2.9308040e-02]
 [1.0544616e-04]
 [6.7148438e-05]]


In [184]:
preds=np.round(preds,3)


In [185]:
test_preds_df=pandas.DataFrame({"prediction":preds.flatten()})
test_preds_df.to_csv("output.csv",index=False)